In [5]:
# read the dictionary file into a list of all words
from resources.rule_definitions import Rule
rule = Rule('resources/twl06_scrabble_dic_american.txt')
words = rule.scrabble_dictionary

In [3]:
# simple EDA
total_words = len(words)
print('Total words:', total_words)

Total words: 178691


In [ ]:
# test simple pattern generator works
from utils.linear.simple_pattern_generator import SimplePatternGenerator

import importlib
import utils
importlib.reload(utils)

spg = SimplePatternGenerator()
spg.generate(pattern='H_T', deck='HTOIAU-', word_list=words)

['HAT', 'HET', 'HIT', 'HOT', 'HUT']

In [ ]:
# test dynamic pattern generator works
from utils.linear.dynamic_pattern_generator import DynamicPatternGenerator

dpg = DynamicPatternGenerator(spg)
dpg.generate(pattern='D(0,2)G', deck='pigsqywozdb-', word_list=words)

['DAG', 'DUG', 'DONG', 'DOG', 'DING', 'DIG']

In [ ]:
# test game state can print pretty board 
from utils.matrix.game_state import Game

game = Game(rule)
game.empty_board()
score = game.new_move([('H', [7, 7]), ('O', [7, 8]), ('T', [7, 9])])
print(score)
game.print_board()

12
        0       1       2       3       4       5       6       7       8       9      10      11      12      13      14
  ---------------------------------------------------------------------------------------------------------
 0 |.TW     .       .       .DL     .       .       .       .TW     .       .       .       .DL     .       .       .TW    
 1 |.       .DW     .       .       .       .TL     .       .       .       .TL     .       .       .       .DW     .      
 2 |.       .       .DW     .       .       .       .DL     .       .DL     .       .       .       .DW     .       .      
 3 |.DL     .       .       .DW     .       .       .       .DL     .       .       .       .DW     .       .       .DL    
 4 |.       .       .       .       .DW     .       .       .       .       .       .DW     .       .       .       .      
 5 |.       .TL     .       .       .       .TL     .       .       .       .TL     .       .       .       .TL     .      
 6 |.       .       .DL

In [8]:
# test longest word user works 
from utils.players.longest_word import OptimiserLength

def test_longest_recommender():
    """Test longest word recommendation"""
    print("="*80)
    
    rule = Rule('resources/twl06_scrabble_dic_american.txt')
    game = Game(rule)
    optimizer = OptimiserLength(rule, game)
    
    # Place multiple words
    game.board[7, 6:10] = list('STAR')
    game.board[5:9, 9] = list('RATE')
    
    deck = list('INGBLEO')
    
    print(f"Deck: {deck}")
    print("\nBoard state:")
    game.print_board()
    
    print("\nRecommendations:")
    recommendations = optimizer.recommend_next_move(deck)
    
    print(f"Found {len(recommendations)} highest-scoring move(s):")
    for i, additions in enumerate(recommendations[:5], 1):
        word_positions = sorted(additions, key=lambda x: (x[1][0], x[1][1]))
        word = ''.join(ch for ch, _ in word_positions)
        positions = [(pos[0], pos[1]) for _, pos in word_positions]
        
        # Determine direction
        if len(set(p[0] for p in positions)) == 1:
            direction = "Horizontal"
        elif len(set(p[1] for p in positions)) == 1:
            direction = "Vertical"
        else:
            direction = "Mixed (error?)"
        
        try:
            score = game.score_calculator(additions)
            print(f"\n  {i}. Word: {word} ({direction})")
            print(f"     Positions: {positions}")
            print(f"     Score: {score}")
            print(f"     Letters placed: {len(additions)}")
        except Exception as e:
            print(f"\n  {i}. Error: {e}")
    
    if len(recommendations) > 5:
        print(f"\n  ... and {len(recommendations) - 5} more")
    print()

test_longest_recommender()


Deck: ['I', 'N', 'G', 'B', 'L', 'E', 'O']

Board state:
        0       1       2       3       4       5       6       7       8       9      10      11      12      13      14
  ---------------------------------------------------------------------------------------------------------
 0 |.TW     .       .       .DL     .       .       .       .TW     .       .       .       .DL     .       .       .TW    
 1 |.       .DW     .       .       .       .TL     .       .       .       .TL     .       .       .       .DW     .      
 2 |.       .       .DW     .       .       .       .DL     .       .DL     .       .       .       .DW     .       .      
 3 |.DL     .       .       .DW     .       .       .       .DL     .       .       .       .DW     .       .       .DL    
 4 |.       .       .       .       .DW     .       .       .       .       .       .DW     .       .       .       .      
 5 |.       .TL     .       .       .       .TL     .       .       .       R(1)TL  .       . 